In [3]:
# Install required packages (run once)
!pip install beautifulsoup4 requests pandas

# Import libraries
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

# Helper functions
def extract_column_from_header(row):
    """Extract column name from HTML table header cell"""
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    column_name = ' '.join(row.contents)
    
    # Filter out digits and empty names
    if not column_name.strip().isdigit():
        return column_name.strip()
    return None

def date_time(table_cells):
    """Extract date and time from table cell"""
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """Extract booster version from table cell"""
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i%2 == 0][0:-1])
    return out

def landing_status(table_cells):
    """Extract landing status from table cell"""
    out = [i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    """Extract payload mass from table cell"""
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0:mass.find("kg")+2]
    else:
        new_mass = 0
    return new_mass

# URL of the Wikipedia page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Request the page
response = requests.get(static_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables
html_tables = soup.find_all('table')

# The third table contains launch records
first_launch_table = html_tables[2]

# Extract column names
column_names = []
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

# Initialize dictionary for launch data
launch_dict = dict.fromkeys(column_names)

# Remove irrelevant column
del launch_dict['Date and time ( )']

# Initialize with empty lists
for key in launch_dict:
    launch_dict[key] = []
    
# Add new columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Parse all launch tables
for table in soup.find_all('table', class_="wikitable plainrowheaders collapsible"):
    for row in table.find_all("tr"):
        # Check for flight number
        if row.th and row.th.string:
            flight_number = row.th.string.strip()
            if flight_number.isdigit():
                # Get all table cells
                cells = row.find_all('td')
                if len(cells) >= 8:  # Ensure we have enough cells
                    # Flight Number
                    launch_dict['Flight No.'].append(flight_number)
                    
                    # Date and Time
                    datatimelist = date_time(cells[0])
                    launch_dict['Date'].append(datatimelist[0].strip(','))
                    launch_dict['Time'].append(datatimelist[1])
                    
                    # Booster Version
                    bv = booster_version(cells[1])
                    if not bv and cells[1].a:
                        bv = cells[1].a.string
                    launch_dict['Version Booster'].append(bv)
                    
                    # Launch Site
                    launch_site = cells[2].a.string if cells[2].a else None
                    launch_dict['Launch site'].append(launch_site)
                    
                    # Payload
                    payload = cells[3].a.string if cells[3].a else None
                    launch_dict['Payload'].append(payload)
                    
                    # Payload Mass
                    payload_mass = get_mass(cells[4])
                    launch_dict['Payload mass'].append(payload_mass)
                    
                    # Orbit
                    orbit = cells[5].a.string if cells[5].a else None
                    launch_dict['Orbit'].append(orbit)
                    
                    # Customer
                    customer = cells[6].a.string if cells[6].a else None
                    launch_dict['Customer'].append(customer)
                    
                    # Launch Outcome
                    launch_outcome = list(cells[7].strings)[0] if cells[7].strings else None
                    launch_dict['Launch outcome'].append(launch_outcome)
                    
                    # Booster Landing
                    booster_landing = landing_status(cells[8]) if len(cells) > 8 else None
                    launch_dict['Booster landing'].append(booster_landing)

# Create DataFrame
df = pd.DataFrame(launch_dict)

# Save to CSV
df.to_csv('spacex_web_scraped.csv', index=False)

print("Scraping completed successfully!")
print(f"Found {len(df)} launch records")

Scraping completed successfully!
Found 121 launch records
